# Distribución de rentas por origen

En este cuaderno tratamos los datos del INE que tratan de la distribución de la renta por origen (salarios, pensiones, desempleo, etc...). Al igual que en el caso de las rentas bruta y neta, se trata de una estadistica experimental, y el formato no es el más cómodo.  

Procederemos de la misma forma que el cuaderno de rentas INE anterior a la hora de tratar los datos que faltan: si existen aquellos a nivel de municipio rellenaremos con ellos los de las secciones; si no hay siguiera los primeros utilizaremos la media de toda la columna.

Aquí directamente no contamos con datos del País Vasco o Navarra en absoluto, por lo que habrá que tratar esta falta de ellos a la hora de hacer los modelos.

Comenzamos cargando las librerías y el fichero excel, que hubo que tratar ligeramente quitando filas iniciales y la fusión de las celdas de los encabezados.

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rentas = pd.read_excel('/content/drive/MyDrive/Proyecto_KeepCoding - Propio/Data/Dist_renta_hogares_ref.xls')

Vemos que, al menos, hay columnas que parecen bastante rellenas, a priori, otras no.

In [ ]:
rentas.head()

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015
0,11903 San Martín del Tesorillo,4683,,,,2017,,,,320,,,,571,,,,602,,,
1,1190301 San Martín del Tesorillo distrito 01,4683,,,,2017,,,,320,,,,571,,,,602,,,
2,1190301001 San Martín del Tesorillo sección 01001,4903,,,,1959,,,,321,,,,587,,,,591,,,
3,1190301002 San Martín del Tesorillo sección 01002,4396,,,,2093,,,,318,,,,549,,,,617,,,
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,


In [ ]:
rentas.shape

(51744, 21)

Procedemos a crear la columna con los datos de cada territorio.

In [ ]:
rentas['Código'] = rentas['Territorio'].str.split(pat = ' ', n = 1)

In [ ]:
rentas.head()

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código
0,11903 San Martín del Tesorillo,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]"
1,1190301 San Martín del Tesorillo distrito 01,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[1190301, San Martín del Tesorillo distrito 01]"
2,1190301001 San Martín del Tesorillo sección 01001,4903,,,,1959,,,,321,,,,587,,,,591,,,,"[1190301001, San Martín del Tesorillo sección ..."
3,1190301002 San Martín del Tesorillo sección 01002,4396,,,,2093,,,,318,,,,549,,,,617,,,,"[1190301002, San Martín del Tesorillo sección ..."
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]"


In [ ]:
rentas.shape

(51744, 22)

In [ ]:
rentas['Código'][125][0]

'0200306009'

Vemos que en algunos casos, las columnas están todas ellas rellenas; es el caso parecido al fichero de las rentas.

In [ ]:
rentas.tail()

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código
51739,5200108011 Melilla sección 08011,11378,10935,11094,11119,1508,1275,1402,1274,167,154,188,166,646,576,516,506,5148,5919,5596,5104,"[5200108011, Melilla sección 08011]"
51740,5200108012 Melilla sección 08012,13272,13218,12951,13038,2763,2786,2836,2445,169,180,188,177,723,699,666,651,3130,3026,3004,2968,"[5200108012, Melilla sección 08012]"
51741,5200108013 Melilla sección 08013,10102,10066,9841,9640,1807,1685,1997,1615,234,225,280,252,627,548,591,509,1373,1628,1615,1295,"[5200108013, Melilla sección 08013]"
51742,5200108014 Melilla sección 08014,5923,5647,5970,6061,2463,2438,2220,2136,244,260,289,284,926,950,860,854,407,356,289,369,"[5200108014, Melilla sección 08014]"
51743,5200108015 Melilla sección 08015,10651,9934,9941,9570,1724,1599,1656,1638,203,221,244,218,813,695,600,555,1515,1349,1328,1265,"[5200108015, Melilla sección 08015]"


In [ ]:
rentas.dtypes

Territorio      object
Sal 2018        object
Sal 2017        object
Sal 2016        object
Sal 2015        object
Pen 2018        object
Pen 2017        object
Pen 2016        object
Pen 2015        object
Desem 2018      object
Desem 2017      object
Desem 2016      object
Desem 2015      object
Ot pres 2018    object
Ot pres 2017    object
Ot pres 2016    object
Ot pres 2015    object
Ot ing 2018     object
Ot ing 2017     object
Ot ing 2016     object
Ot ing 2015     object
Código          object
dtype: object

Extraemos el código numérico del territorio, aunque ya vemos que el dataframe contiene errores, como en el registro de índice 8.

In [ ]:
rentas['Código1'] = ''

In [ ]:
rentas['Código1'] = rentas['Código'].apply(lambda x : x[0])

In [ ]:
rentas[4:10]

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901
5,1490101 Fuente Carreteros distrito 01,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[1490101, Fuente Carreteros distrito 01]",1490101
6,1490101001 Fuente Carreteros sección 01001,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[1490101001, Fuente Carreteros sección 01001]",1490101001
7,"14902 Guijarrosa, La",4288,,,,2201,,,,846,,,,361,,,,682,,,,"[14902, Guijarrosa, La]",14902
8,"Guijarrosa, La distrito 01",4288,,,,2201,,,,846,,,,361,,,,682,,,,"[Guijarrosa,, La distrito 01]","Guijarrosa,"
9,"1490201001 Guijarrosa, La sección 01001",4288,,,,2201,,,,846,,,,361,,,,682,,,,"[1490201001, Guijarrosa, La sección 01001]",1490201001


In [ ]:
rentas.iloc[6,5]

2267

Al igual que hicimos en el otro cuaderno, almacenamos el código del municipio de cada registro, que utilizarmos después para hacer el merge.

In [ ]:
rentas['cod_aux'] = rentas['Código1'].str[0:5]

In [ ]:
rentas

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux
0,11903 San Martín del Tesorillo,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903
1,1190301 San Martín del Tesorillo distrito 01,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[1190301, San Martín del Tesorillo distrito 01]",1190301,11903
2,1190301001 San Martín del Tesorillo sección 01001,4903,,,,1959,,,,321,,,,587,,,,591,,,,"[1190301001, San Martín del Tesorillo sección ...",1190301001,11903
3,1190301002 San Martín del Tesorillo sección 01002,4396,,,,2093,,,,318,,,,549,,,,617,,,,"[1190301002, San Martín del Tesorillo sección ...",1190301002,11903
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51739,5200108011 Melilla sección 08011,11378,10935,11094,11119,1508,1275,1402,1274,167,154,188,166,646,576,516,506,5148,5919,5596,5104,"[5200108011, Melilla sección 08011]",5200108011,52001
51740,5200108012 Melilla sección 08012,13272,13218,12951,13038,2763,2786,2836,2445,169,180,188,177,723,699,666,651,3130,3026,3004,2968,"[5200108012, Melilla sección 08012]",5200108012,52001
51741,5200108013 Melilla sección 08013,10102,10066,9841,9640,1807,1685,1997,1615,234,225,280,252,627,548,591,509,1373,1628,1615,1295,"[5200108013, Melilla sección 08013]",5200108013,52001
51742,5200108014 Melilla sección 08014,5923,5647,5970,6061,2463,2438,2220,2136,244,260,289,284,926,950,860,854,407,356,289,369,"[5200108014, Melilla sección 08014]",5200108014,52001


Vuelven a aparecer puntos y espacios en blanco en las columnas:

In [ ]:
rentas['Sal 2018'].value_counts()

.        3955
          286
26674     133
1767       42
6261       22
         ... 
11901       1
11902       1
11904       1
11906       1
16390       1
Name: Sal 2018, Length: 11771, dtype: int64

Seleccionamos los datos de los municipios, sabiendo que el cógido del territorio debe tener cinco caracteres. Vemos que hay municipios que tienen solo parte de los datos.

In [ ]:
rentas_muni = rentas.loc[rentas['Código1'].str.len() == 5]

In [ ]:
rentas_muni

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux
0,11903 San Martín del Tesorillo,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901
7,"14902 Guijarrosa, La",4288,,,,2201,,,,846,,,,361,,,,682,,,,"[14902, Guijarrosa, La]",14902,14902
10,18077 Fornes,4170,,,,3049,,,,813,,,,852,,,,834,,,,"[18077, Fornes]",18077,18077
13,18916 Torrenueva Costa,4422,,,,2095,,,,503,,,,747,,,,1228,,,,"[18916, Torrenueva Costa]",18916,18916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51618,50901 Biel,13108,10463,10204,9636,7146,6922,7679,7398,145,254,257,214,317,331,431,388,2760,2104,2656,2450,"[50901, Biel]",50901,50901
51621,50902 Marracos,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,"[50902, Marracos]",50902,50902
51624,50903 Villamayor de Gállego,10159,10046,9400,9035,3579,3433,3165,3087,184,143,201,247,439,401,372,391,1660,1570,1650,1732,"[50903, Villamayor de Gállego]",50903,50903
51628,51001 Ceuta,8573,8203,8115,8097,1879,1804,1809,1773,312,314,321,339,548,484,437,419,1371,1290,1307,1389,"[51001, Ceuta]",51001,51001


In [ ]:
rentas_muni['Sal 2018'].value_counts()

.       1301
           9
6261       6
5355       6
6150       6
        ... 
4571       1
7211       1
4566       1
4564       1
8191       1
Name: Sal 2018, Length: 4332, dtype: int64

In [ ]:
rentas_muni['Sal 2015'].value_counts()

.       1232
          13
6507       7
3737       6
3999       6
        ... 
5191       1
5190       1
5189       1
5187       1
8193       1
Name: Sal 2015, Length: 4284, dtype: int64

Renombramos la columna del territorio para hacer luego el merge

In [ ]:
rentas_muni.rename(columns={'Código1' : 'cod_aux'})

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,cod_aux,cod_aux
0,11903 San Martín del Tesorillo,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901
7,"14902 Guijarrosa, La",4288,,,,2201,,,,846,,,,361,,,,682,,,,"[14902, Guijarrosa, La]",14902,14902
10,18077 Fornes,4170,,,,3049,,,,813,,,,852,,,,834,,,,"[18077, Fornes]",18077,18077
13,18916 Torrenueva Costa,4422,,,,2095,,,,503,,,,747,,,,1228,,,,"[18916, Torrenueva Costa]",18916,18916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51618,50901 Biel,13108,10463,10204,9636,7146,6922,7679,7398,145,254,257,214,317,331,431,388,2760,2104,2656,2450,"[50901, Biel]",50901,50901
51621,50902 Marracos,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,"[50902, Marracos]",50902,50902
51624,50903 Villamayor de Gállego,10159,10046,9400,9035,3579,3433,3165,3087,184,143,201,247,439,401,372,391,1660,1570,1650,1732,"[50903, Villamayor de Gállego]",50903,50903
51628,51001 Ceuta,8573,8203,8115,8097,1879,1804,1809,1773,312,314,321,339,548,484,437,419,1371,1290,1307,1389,"[51001, Ceuta]",51001,51001


In [ ]:
rentas_muni.columns

Index(['Territorio', 'Sal 2018', 'Sal 2017', 'Sal 2016', 'Sal 2015',
       'Pen 2018', 'Pen 2017', 'Pen 2016', 'Pen 2015', 'Desem 2018',
       'Desem 2017', 'Desem 2016', 'Desem 2015', 'Ot pres 2018',
       'Ot pres 2017', 'Ot pres 2016', 'Ot pres 2015', 'Ot ing 2018',
       'Ot ing 2017', 'Ot ing 2016', 'Ot ing 2015', 'Código', 'Código1',
       'cod_aux'],
      dtype='object')

Renombramos las columnas del df de los municipios para que no haya confusión una vez hagamos el merge.

In [ ]:
rentas_muni = rentas_muni.rename(columns={'Sal 2018' : 'renta_salario_2018', 'Sal 2015' : 'renta_salario_2015',
                                          'Pen 2018' : 'renta_pensiones_2018', 'Pen 2015' : 'renta_pensiones_2015',
                                          'Desem 2018' : 'renta_desempleo_2018', 'Desem 2015' : 'renta_desempleo_2015'
                                            })

In [ ]:
rentas_muni

,Territorio,renta_salario_2018,Sal 2017,Sal 2016,renta_salario_2015,renta_pensiones_2018,Pen 2017,Pen 2016,renta_pensiones_2015,renta_desempleo_2018,Desem 2017,Desem 2016,renta_desempleo_2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux
0,11903 San Martín del Tesorillo,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901
7,"14902 Guijarrosa, La",4288,,,,2201,,,,846,,,,361,,,,682,,,,"[14902, Guijarrosa, La]",14902,14902
10,18077 Fornes,4170,,,,3049,,,,813,,,,852,,,,834,,,,"[18077, Fornes]",18077,18077
13,18916 Torrenueva Costa,4422,,,,2095,,,,503,,,,747,,,,1228,,,,"[18916, Torrenueva Costa]",18916,18916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51618,50901 Biel,13108,10463,10204,9636,7146,6922,7679,7398,145,254,257,214,317,331,431,388,2760,2104,2656,2450,"[50901, Biel]",50901,50901
51621,50902 Marracos,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,"[50902, Marracos]",50902,50902
51624,50903 Villamayor de Gállego,10159,10046,9400,9035,3579,3433,3165,3087,184,143,201,247,439,401,372,391,1660,1570,1650,1732,"[50903, Villamayor de Gállego]",50903,50903
51628,51001 Ceuta,8573,8203,8115,8097,1879,1804,1809,1773,312,314,321,339,548,484,437,419,1371,1290,1307,1389,"[51001, Ceuta]",51001,51001


In [ ]:
rentas_muni.columns

Index(['Territorio', 'renta_salario_2018', 'Sal 2017', 'Sal 2016',
       'renta_salario_2015', 'renta_pensiones_2018', 'Pen 2017', 'Pen 2016',
       'renta_pensiones_2015', 'renta_desempleo_2018', 'Desem 2017',
       'Desem 2016', 'renta_desempleo_2015', 'Ot pres 2018', 'Ot pres 2017',
       'Ot pres 2016', 'Ot pres 2015', 'Ot ing 2018', 'Ot ing 2017',
       'Ot ing 2016', 'Ot ing 2015', 'Código', 'Código1', 'cod_aux'],
      dtype='object')

Antes de hacer el merge nos quedamos con las columnas del df de los municipios que realmente nos interesan.

In [ ]:
rentas_muni = rentas_muni[['cod_aux', 'renta_salario_2018', 'renta_salario_2015', 'renta_pensiones_2018',  'renta_pensiones_2015', 'renta_desempleo_2018', 'renta_desempleo_2015'  ]]

In [ ]:
rentas_muni

,cod_aux,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
0,11903,4683,,2017,,320,
4,14901,3669,,2267,,846,
7,14902,4288,,2201,,846,
10,18077,4170,,3049,,813,
13,18916,4422,,2095,,503,
...,...,...,...,...,...,...,...
51618,50901,13108,9636,7146,7398,145,214
51621,50902,.,.,.,.,.,.
51624,50903,10159,9035,3579,3087,184,247
51628,51001,8573,8097,1879,1773,312,339


Finalmente hacemos el merge mediante la columna que identifica el municipio, igual que hicimos en el cuaderno de las rentas.

In [ ]:
rentas_result = pd.merge(rentas, rentas_muni, how="outer", on=['cod_aux'])

In [ ]:
rentas_result

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
0,11903 San Martín del Tesorillo,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903,4683,,2017,,320,
1,1190301 San Martín del Tesorillo distrito 01,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[1190301, San Martín del Tesorillo distrito 01]",1190301,11903,4683,,2017,,320,
2,1190301001 San Martín del Tesorillo sección 01001,4903,,,,1959,,,,321,,,,587,,,,591,,,,"[1190301001, San Martín del Tesorillo sección ...",1190301001,11903,4683,,2017,,320,
3,1190301002 San Martín del Tesorillo sección 01002,4396,,,,2093,,,,318,,,,549,,,,617,,,,"[1190301002, San Martín del Tesorillo sección ...",1190301002,11903,4683,,2017,,320,
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901,3669,,2267,,846,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51739,5200108011 Melilla sección 08011,11378,10935,11094,11119,1508,1275,1402,1274,167,154,188,166,646,576,516,506,5148,5919,5596,5104,"[5200108011, Melilla sección 08011]",5200108011,52001,7949,7359,1610,1476,243,275
51740,5200108012 Melilla sección 08012,13272,13218,12951,13038,2763,2786,2836,2445,169,180,188,177,723,699,666,651,3130,3026,3004,2968,"[5200108012, Melilla sección 08012]",5200108012,52001,7949,7359,1610,1476,243,275
51741,5200108013 Melilla sección 08013,10102,10066,9841,9640,1807,1685,1997,1615,234,225,280,252,627,548,591,509,1373,1628,1615,1295,"[5200108013, Melilla sección 08013]",5200108013,52001,7949,7359,1610,1476,243,275
51742,5200108014 Melilla sección 08014,5923,5647,5970,6061,2463,2438,2220,2136,244,260,289,284,926,950,860,854,407,356,289,369,"[5200108014, Melilla sección 08014]",5200108014,52001,7949,7359,1610,1476,243,275


In [ ]:
rentas_result.columns

Index(['Territorio', 'Sal 2018', 'Sal 2017', 'Sal 2016', 'Sal 2015',
       'Pen 2018', 'Pen 2017', 'Pen 2016', 'Pen 2015', 'Desem 2018',
       'Desem 2017', 'Desem 2016', 'Desem 2015', 'Ot pres 2018',
       'Ot pres 2017', 'Ot pres 2016', 'Ot pres 2015', 'Ot ing 2018',
       'Ot ing 2017', 'Ot ing 2016', 'Ot ing 2015', 'Código', 'Código1',
       'cod_aux', 'renta_salario_2018', 'renta_salario_2015',
       'renta_pensiones_2018', 'renta_pensiones_2015', 'renta_desempleo_2018',
       'renta_desempleo_2015'],
      dtype='object')

Comenzamos a rellenar los registros de las secciones que no tienen datos, pero que sí existen para su municipio, en este caso la columna del salario en 2018.

In [ ]:
for ind in range(len(rentas_result['Sal 2018'])):
  if len(str(rentas_result['Sal 2018'][ind])) < 3:
    rentas_result['Sal 2018'][ind] = rentas_result['renta_salario_2018'][ind]

In [ ]:
rentas_result[0:10]

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
0,11903 San Martín del Tesorillo,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903,4683,,2017,,320,
1,1190301 San Martín del Tesorillo distrito 01,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[1190301, San Martín del Tesorillo distrito 01]",1190301,11903,4683,,2017,,320,
2,1190301001 San Martín del Tesorillo sección 01001,4903,,,,1959,,,,321,,,,587,,,,591,,,,"[1190301001, San Martín del Tesorillo sección ...",1190301001,11903,4683,,2017,,320,
3,1190301002 San Martín del Tesorillo sección 01002,4396,,,,2093,,,,318,,,,549,,,,617,,,,"[1190301002, San Martín del Tesorillo sección ...",1190301002,11903,4683,,2017,,320,
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901,3669,,2267,,846,
5,1490101 Fuente Carreteros distrito 01,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[1490101, Fuente Carreteros distrito 01]",1490101,14901,3669,,2267,,846,
6,1490101001 Fuente Carreteros sección 01001,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[1490101001, Fuente Carreteros sección 01001]",1490101001,14901,3669,,2267,,846,
7,"14902 Guijarrosa, La",4288,,,,2201,,,,846,,,,361,,,,682,,,,"[14902, Guijarrosa, La]",14902,14902,4288,,2201,,846,
8,"1490201001 Guijarrosa, La sección 01001",4288,,,,2201,,,,846,,,,361,,,,682,,,,"[1490201001, Guijarrosa, La sección 01001]",1490201001,14902,4288,,2201,,846,
9,"Guijarrosa, La distrito 01",4288,,,,2201,,,,846,,,,361,,,,682,,,,"[Guijarrosa,, La distrito 01]","Guijarrosa,",Guija,NaN,NaN,NaN,NaN,NaN,NaN


Apliacamos el mismo procedimiento a otras cinco columnas. Reiteramos que no es el método más pythonista, pero preferimos ir paso a paso. Las seis columnas tratadas son las que nos interesan.

In [ ]:
for ind in range(len(rentas_result['Sal 2015'])):
  if len(str(rentas_result['Sal 2015'][ind])) < 3:
    rentas_result['Sal 2015'][ind] = rentas_result['renta_salario_2015'][ind]

In [ ]:
for ind in range(len(rentas_result['Pen 2018'])):
  if len(str(rentas_result['Pen 2018'][ind])) < 3:
    rentas_result['Pen 2018'][ind] = rentas_result['renta_pensiones_2018'][ind]

In [ ]:
for ind in range(len(rentas_result['Pen 2015'])):
  if len(str(rentas_result['Pen 2015'][ind])) < 3:
    rentas_result['Pen 2015'][ind] = rentas_result['renta_pensiones_2015'][ind]

In [ ]:
for ind in range(len(rentas_result['Desem 2018'])):
  if len(str(rentas_result['Desem 2018'][ind])) < 3:
    rentas_result['Desem 2018'][ind] = rentas_result['renta_desempleo_2018'][ind]

In [ ]:
for ind in range(len(rentas_result['Desem 2015'])):
  if len(str(rentas_result['Desem 2015'][ind])) < 3:
    rentas_result['Desem 2015'][ind] = rentas_result['renta_desempleo_2015'][ind]

In [ ]:
rentas_result

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
0,11903 San Martín del Tesorillo,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903,4683,,2017,,320,
1,1190301 San Martín del Tesorillo distrito 01,4683,,,,2017,,,,320,,,,571,,,,602,,,,"[1190301, San Martín del Tesorillo distrito 01]",1190301,11903,4683,,2017,,320,
2,1190301001 San Martín del Tesorillo sección 01001,4903,,,,1959,,,,321,,,,587,,,,591,,,,"[1190301001, San Martín del Tesorillo sección ...",1190301001,11903,4683,,2017,,320,
3,1190301002 San Martín del Tesorillo sección 01002,4396,,,,2093,,,,318,,,,549,,,,617,,,,"[1190301002, San Martín del Tesorillo sección ...",1190301002,11903,4683,,2017,,320,
4,14901 Fuente Carreteros,3669,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901,3669,,2267,,846,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51739,5200108011 Melilla sección 08011,11378,10935,11094,11119,1508,1275,1402,1274,167,154,188,166,646,576,516,506,5148,5919,5596,5104,"[5200108011, Melilla sección 08011]",5200108011,52001,7949,7359,1610,1476,243,275
51740,5200108012 Melilla sección 08012,13272,13218,12951,13038,2763,2786,2836,2445,169,180,188,177,723,699,666,651,3130,3026,3004,2968,"[5200108012, Melilla sección 08012]",5200108012,52001,7949,7359,1610,1476,243,275
51741,5200108013 Melilla sección 08013,10102,10066,9841,9640,1807,1685,1997,1615,234,225,280,252,627,548,591,509,1373,1628,1615,1295,"[5200108013, Melilla sección 08013]",5200108013,52001,7949,7359,1610,1476,243,275
51742,5200108014 Melilla sección 08014,5923,5647,5970,6061,2463,2438,2220,2136,244,260,289,284,926,950,860,854,407,356,289,369,"[5200108014, Melilla sección 08014]",5200108014,52001,7949,7359,1610,1476,243,275


Ahora tratamos los registros para las que no hay siquiera datos de los municipios. Utilizaremos la media de la columna entera.

Vemos que en muchos casos existen puntos en lugar de datos numéricos, y que todas las columnas son de tipo objeto.

In [ ]:
rentas_result['Sal 2018'].value_counts()

.        3903
26674     133
1767       42
           25
6858       23
         ... 
11902       1
11904       1
11906       1
11909       1
16390       1
Name: Sal 2018, Length: 11771, dtype: int64

In [ ]:
rentas_result.dtypes

Territorio              object
Sal 2018                object
Sal 2017                object
Sal 2016                object
Sal 2015                object
Pen 2018                object
Pen 2017                object
Pen 2016                object
Pen 2015                object
Desem 2018              object
Desem 2017              object
Desem 2016              object
Desem 2015              object
Ot pres 2018            object
Ot pres 2017            object
Ot pres 2016            object
Ot pres 2015            object
Ot ing 2018             object
Ot ing 2017             object
Ot ing 2016             object
Ot ing 2015             object
Código                  object
Código1                 object
cod_aux                 object
renta_salario_2018      object
renta_salario_2015      object
renta_pensiones_2018    object
renta_pensiones_2015    object
renta_desempleo_2018    object
renta_desempleo_2015    object
dtype: object

Procedemos igual que hicimos en el cuaderno de las rentas: empezamos por sustituir los espacios en blanco y los puntos por ceros.

Después pasamos la columna a tipo numérico.

In [ ]:
rentas_result['Sal 2018'] = rentas_result['Sal 2018'].replace(' ', 0)
rentas_result['Sal 2018'] = rentas_result['Sal 2018'].replace('.', 0)


In [ ]:
rentas_result['Sal 2018'] = pd.to_numeric(rentas_result['Sal 2018'])

Ahora calculamos la media de la columna excluyendo los ceros que acabamos de introducir, y mediante una función introducimos este valor precisamente donde hemos puesto justo antes ceros.

In [ ]:
avg = rentas_result[rentas_result['Sal 2018'] > 1]['Sal 2018'].mean()

In [ ]:
avg

7855.336602810775

In [ ]:
rentas_result['Sal 2018'] = rentas_result['Sal 2018'].apply(celda)

In [ ]:
def celda(x):
  if x == 0:
    return avg
  else:
    return x

In [ ]:
avg

7855.336602810775

In [ ]:
rentas_result

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
0,11903 San Martín del Tesorillo,4683.0,,,,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903,4683,,2017,,320,
1,1190301 San Martín del Tesorillo distrito 01,4683.0,,,,2017,,,,320,,,,571,,,,602,,,,"[1190301, San Martín del Tesorillo distrito 01]",1190301,11903,4683,,2017,,320,
2,1190301001 San Martín del Tesorillo sección 01001,4903.0,,,,1959,,,,321,,,,587,,,,591,,,,"[1190301001, San Martín del Tesorillo sección ...",1190301001,11903,4683,,2017,,320,
3,1190301002 San Martín del Tesorillo sección 01002,4396.0,,,,2093,,,,318,,,,549,,,,617,,,,"[1190301002, San Martín del Tesorillo sección ...",1190301002,11903,4683,,2017,,320,
4,14901 Fuente Carreteros,3669.0,,,,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901,3669,,2267,,846,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51739,5200108011 Melilla sección 08011,11378.0,10935,11094,11119,1508,1275,1402,1274,167,154,188,166,646,576,516,506,5148,5919,5596,5104,"[5200108011, Melilla sección 08011]",5200108011,52001,7949,7359,1610,1476,243,275
51740,5200108012 Melilla sección 08012,13272.0,13218,12951,13038,2763,2786,2836,2445,169,180,188,177,723,699,666,651,3130,3026,3004,2968,"[5200108012, Melilla sección 08012]",5200108012,52001,7949,7359,1610,1476,243,275
51741,5200108013 Melilla sección 08013,10102.0,10066,9841,9640,1807,1685,1997,1615,234,225,280,252,627,548,591,509,1373,1628,1615,1295,"[5200108013, Melilla sección 08013]",5200108013,52001,7949,7359,1610,1476,243,275
51742,5200108014 Melilla sección 08014,5923.0,5647,5970,6061,2463,2438,2220,2136,244,260,289,284,926,950,860,854,407,356,289,369,"[5200108014, Melilla sección 08014]",5200108014,52001,7949,7359,1610,1476,243,275


In [ ]:
len(rentas_result['Sal 2018'])

51744

Apliacamos el mismo proceder a la columna 'Sal 2015', y se muestra para los primeros registros el resultado.

In [ ]:
rentas_result['Sal 2015'] = rentas_result['Sal 2015'].replace(' ', 0)
rentas_result['Sal 2015'] = rentas_result['Sal 2015'].replace('.', 0)
rentas_result['Sal 2015'] = pd.to_numeric(rentas_result['Sal 2015'])
avg = rentas_result[rentas_result['Sal 2015'] > 1]['Sal 2015'].mean()
rentas_result['Sal 2015'] = rentas_result['Sal 2015'].apply(celda)

In [ ]:
rentas_result

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
0,11903 San Martín del Tesorillo,4683.0,,,6845.948425,2017,,,,320,,,,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903,4683,,2017,,320,
1,1190301 San Martín del Tesorillo distrito 01,4683.0,,,6845.948425,2017,,,,320,,,,571,,,,602,,,,"[1190301, San Martín del Tesorillo distrito 01]",1190301,11903,4683,,2017,,320,
2,1190301001 San Martín del Tesorillo sección 01001,4903.0,,,6845.948425,1959,,,,321,,,,587,,,,591,,,,"[1190301001, San Martín del Tesorillo sección ...",1190301001,11903,4683,,2017,,320,
3,1190301002 San Martín del Tesorillo sección 01002,4396.0,,,6845.948425,2093,,,,318,,,,549,,,,617,,,,"[1190301002, San Martín del Tesorillo sección ...",1190301002,11903,4683,,2017,,320,
4,14901 Fuente Carreteros,3669.0,,,6845.948425,2267,,,,846,,,,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901,3669,,2267,,846,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51739,5200108011 Melilla sección 08011,11378.0,10935,11094,11119.000000,1508,1275,1402,1274,167,154,188,166,646,576,516,506,5148,5919,5596,5104,"[5200108011, Melilla sección 08011]",5200108011,52001,7949,7359,1610,1476,243,275
51740,5200108012 Melilla sección 08012,13272.0,13218,12951,13038.000000,2763,2786,2836,2445,169,180,188,177,723,699,666,651,3130,3026,3004,2968,"[5200108012, Melilla sección 08012]",5200108012,52001,7949,7359,1610,1476,243,275
51741,5200108013 Melilla sección 08013,10102.0,10066,9841,9640.000000,1807,1685,1997,1615,234,225,280,252,627,548,591,509,1373,1628,1615,1295,"[5200108013, Melilla sección 08013]",5200108013,52001,7949,7359,1610,1476,243,275
51742,5200108014 Melilla sección 08014,5923.0,5647,5970,6061.000000,2463,2438,2220,2136,244,260,289,284,926,950,860,854,407,356,289,369,"[5200108014, Melilla sección 08014]",5200108014,52001,7949,7359,1610,1476,243,275


Aplicamos lo mismo a las otras cuatro columnas que nos interesan. No muy pythonista, lo reconocemos, pero casi mejor ir con los pies de plomo.

In [ ]:
rentas_result['Pen 2018'] = rentas_result['Pen 2018'].replace(' ', 0)
rentas_result['Pen 2018'] = rentas_result['Pen 2018'].replace('.', 0)
rentas_result['Pen 2018'] = pd.to_numeric(rentas_result['Pen 2018'])
avg = rentas_result[rentas_result['Pen 2018'] > 1]['Pen 2018'].mean()
rentas_result['Pen 2018'] = rentas_result['Pen 2018'].apply(celda)

In [ ]:
rentas_result['Pen 2015'] = rentas_result['Pen 2015'].replace(' ', 0)
rentas_result['Pen 2015'] = rentas_result['Pen 2015'].replace('.', 0)
rentas_result['Pen 2015'] = pd.to_numeric(rentas_result['Pen 2015'])
avg = rentas_result[rentas_result['Pen 2015'] > 1]['Pen 2015'].mean()
rentas_result['Pen 2015'] = rentas_result['Pen 2015'].apply(celda)

In [ ]:
rentas_result['Desem 2018'] = rentas_result['Desem 2018'].replace(' ', 0)
rentas_result['Desem 2018'] = rentas_result['Desem 2018'].replace('.', 0)
rentas_result['Desem 2018'] = pd.to_numeric(rentas_result['Desem 2018'])
avg = rentas_result[rentas_result['Desem 2018'] > 1]['Desem 2018'].mean()
rentas_result['Desem 2018'] = rentas_result['Desem 2018'].apply(celda)

In [ ]:
rentas_result['Desem 2015'] = rentas_result['Desem 2015'].replace(' ', 0)
rentas_result['Desem 2015'] = rentas_result['Desem 2015'].replace('.', 0)
rentas_result['Desem 2015'] = pd.to_numeric(rentas_result['Desem 2015'])
avg = rentas_result[rentas_result['Desem 2015'] > 1]['Desem 2015'].mean()
rentas_result['Desem 2015'] = rentas_result['Desem 2015'].apply(celda)

Ya parece que tenemos las seis columnas rellenas. Son la distribución de la renta en 2015 y 2018 según salario, pensiones y desempleo.

In [ ]:
rentas_result

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
0,11903 San Martín del Tesorillo,4683.0,,,6845.948425,2017.0,,,2985.302533,320.0,,,347.217589,571,,,,602,,,,"[11903, San Martín del Tesorillo]",11903,11903,4683,,2017,,320,
1,1190301 San Martín del Tesorillo distrito 01,4683.0,,,6845.948425,2017.0,,,2985.302533,320.0,,,347.217589,571,,,,602,,,,"[1190301, San Martín del Tesorillo distrito 01]",1190301,11903,4683,,2017,,320,
2,1190301001 San Martín del Tesorillo sección 01001,4903.0,,,6845.948425,1959.0,,,2985.302533,321.0,,,347.217589,587,,,,591,,,,"[1190301001, San Martín del Tesorillo sección ...",1190301001,11903,4683,,2017,,320,
3,1190301002 San Martín del Tesorillo sección 01002,4396.0,,,6845.948425,2093.0,,,2985.302533,318.0,,,347.217589,549,,,,617,,,,"[1190301002, San Martín del Tesorillo sección ...",1190301002,11903,4683,,2017,,320,
4,14901 Fuente Carreteros,3669.0,,,6845.948425,2267.0,,,2985.302533,846.0,,,347.217589,548,,,,604,,,,"[14901, Fuente Carreteros]",14901,14901,3669,,2267,,846,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51739,5200108011 Melilla sección 08011,11378.0,10935,11094,11119.000000,1508.0,1275,1402,1274.000000,167.0,154,188,166.000000,646,576,516,506,5148,5919,5596,5104,"[5200108011, Melilla sección 08011]",5200108011,52001,7949,7359,1610,1476,243,275
51740,5200108012 Melilla sección 08012,13272.0,13218,12951,13038.000000,2763.0,2786,2836,2445.000000,169.0,180,188,177.000000,723,699,666,651,3130,3026,3004,2968,"[5200108012, Melilla sección 08012]",5200108012,52001,7949,7359,1610,1476,243,275
51741,5200108013 Melilla sección 08013,10102.0,10066,9841,9640.000000,1807.0,1685,1997,1615.000000,234.0,225,280,252.000000,627,548,591,509,1373,1628,1615,1295,"[5200108013, Melilla sección 08013]",5200108013,52001,7949,7359,1610,1476,243,275
51742,5200108014 Melilla sección 08014,5923.0,5647,5970,6061.000000,2463.0,2438,2220,2136.000000,244.0,260,289,284.000000,926,950,860,854,407,356,289,369,"[5200108014, Melilla sección 08014]",5200108014,52001,7949,7359,1610,1476,243,275


Ahora nos quedamos con los datos de las secciones, sabiendo que el código de estos territorios tiene 10 caracteres. 

Vemos que hay solo 34 mil registros, no los habituales 36 mil, pero hay que tener en cuenta que no tenemos datos del País Vasco o Navarra.

In [ ]:
rentas_result_sec = rentas_result.loc[rentas_result['Código1'].str.len() == 10]

In [ ]:
rentas_result_sec

,Territorio,Sal 2018,Sal 2017,Sal 2016,Sal 2015,Pen 2018,Pen 2017,Pen 2016,Pen 2015,Desem 2018,Desem 2017,Desem 2016,Desem 2015,Ot pres 2018,Ot pres 2017,Ot pres 2016,Ot pres 2015,Ot ing 2018,Ot ing 2017,Ot ing 2016,Ot ing 2015,Código,Código1,cod_aux,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
2,1190301001 San Martín del Tesorillo sección 01001,4903.0,,,6845.948425,1959.0,,,2985.302533,321.0,,,347.217589,587,,,,591,,,,"[1190301001, San Martín del Tesorillo sección ...",1190301001,11903,4683,,2017,,320,
3,1190301002 San Martín del Tesorillo sección 01002,4396.0,,,6845.948425,2093.0,,,2985.302533,318.0,,,347.217589,549,,,,617,,,,"[1190301002, San Martín del Tesorillo sección ...",1190301002,11903,4683,,2017,,320,
6,1490101001 Fuente Carreteros sección 01001,3669.0,,,6845.948425,2267.0,,,2985.302533,846.0,,,347.217589,548,,,,604,,,,"[1490101001, Fuente Carreteros sección 01001]",1490101001,14901,3669,,2267,,846,
8,"1490201001 Guijarrosa, La sección 01001",4288.0,,,6845.948425,2201.0,,,2985.302533,846.0,,,347.217589,361,,,,682,,,,"[1490201001, Guijarrosa, La sección 01001]",1490201001,14902,4288,,2201,,846,
12,1807701001 Fornes sección 01001,4170.0,,,6845.948425,3049.0,,,2985.302533,813.0,,,347.217589,852,,,,834,,,,"[1807701001, Fornes sección 01001]",1807701001,18077,4170,,3049,,813,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51739,5200108011 Melilla sección 08011,11378.0,10935,11094,11119.000000,1508.0,1275,1402,1274.000000,167.0,154,188,166.000000,646,576,516,506,5148,5919,5596,5104,"[5200108011, Melilla sección 08011]",5200108011,52001,7949,7359,1610,1476,243,275
51740,5200108012 Melilla sección 08012,13272.0,13218,12951,13038.000000,2763.0,2786,2836,2445.000000,169.0,180,188,177.000000,723,699,666,651,3130,3026,3004,2968,"[5200108012, Melilla sección 08012]",5200108012,52001,7949,7359,1610,1476,243,275
51741,5200108013 Melilla sección 08013,10102.0,10066,9841,9640.000000,1807.0,1685,1997,1615.000000,234.0,225,280,252.000000,627,548,591,509,1373,1628,1615,1295,"[5200108013, Melilla sección 08013]",5200108013,52001,7949,7359,1610,1476,243,275
51742,5200108014 Melilla sección 08014,5923.0,5647,5970,6061.000000,2463.0,2438,2220,2136.000000,244.0,260,289,284.000000,926,950,860,854,407,356,289,369,"[5200108014, Melilla sección 08014]",5200108014,52001,7949,7359,1610,1476,243,275


Cambiamos los nombres de varias columnas para hacerlas más reconocibles y útiles cuando hagamos la unificación de los datos.

In [ ]:
rentas_result_sec = rentas_result_sec.rename(columns = {'Código1': 'cod_sec'})

In [ ]:
rentas_result_sec.columns

Index(['Territorio', 'Sal 2018', 'Sal 2017', 'Sal 2016', 'Sal 2015',
       'Pen 2018', 'Pen 2017', 'Pen 2016', 'Pen 2015', 'Desem 2018',
       'Desem 2017', 'Desem 2016', 'Desem 2015', 'Ot pres 2018',
       'Ot pres 2017', 'Ot pres 2016', 'Ot pres 2015', 'Ot ing 2018',
       'Ot ing 2017', 'Ot ing 2016', 'Ot ing 2015', 'Código', 'cod_sec',
       'cod_aux', 'renta_salario_2018', 'renta_salario_2015',
       'renta_pensiones_2018', 'renta_pensiones_2015', 'renta_desempleo_2018',
       'renta_desempleo_2015'],
      dtype='object')

In [ ]:
rentas_result_sec = rentas_result_sec[['Territorio', 'cod_sec', 'Sal 2018', 'Sal 2015', 'Pen 2018', 'Pen 2015',
       'Desem 2018', 'Desem 2015']]

In [ ]:
rentas_result_sec

,Territorio,cod_sec,Sal 2018,Sal 2015,Pen 2018,Pen 2015,Desem 2018,Desem 2015
2,1190301001 San Martín del Tesorillo sección 01001,1190301001,4903.0,6845.948425,1959.0,2985.302533,321.0,347.217589
3,1190301002 San Martín del Tesorillo sección 01002,1190301002,4396.0,6845.948425,2093.0,2985.302533,318.0,347.217589
6,1490101001 Fuente Carreteros sección 01001,1490101001,3669.0,6845.948425,2267.0,2985.302533,846.0,347.217589
8,"1490201001 Guijarrosa, La sección 01001",1490201001,4288.0,6845.948425,2201.0,2985.302533,846.0,347.217589
12,1807701001 Fornes sección 01001,1807701001,4170.0,6845.948425,3049.0,2985.302533,813.0,347.217589
...,...,...,...,...,...,...,...,...
51739,5200108011 Melilla sección 08011,5200108011,11378.0,11119.000000,1508.0,1274.000000,167.0,166.000000
51740,5200108012 Melilla sección 08012,5200108012,13272.0,13038.000000,2763.0,2445.000000,169.0,177.000000
51741,5200108013 Melilla sección 08013,5200108013,10102.0,9640.000000,1807.0,1615.000000,234.0,252.000000
51742,5200108014 Melilla sección 08014,5200108014,5923.0,6061.000000,2463.0,2136.000000,244.0,284.000000


Finalmente guardamos el dataframe, que no depende de las elecciones, y que por lo tanto es único.

In [ ]:
rentas_result_sec.to_csv('/content/drive/MyDrive/Proyecto_KeepCoding (1)/Data/Dist_renta_hogares_15_18.txt', sep = ',', index = False)

Nos queda el dataframe de los datos de los municipios que quizás algún día nos sea útil, por lo que lo guardamos también.

In [ ]:
rentas_muni

,cod_aux,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
0,11903,4683,,2017,,320,
4,14901,3669,,2267,,846,
7,14902,4288,,2201,,846,
10,18077,4170,,3049,,813,
13,18916,4422,,2095,,503,
...,...,...,...,...,...,...,...
51618,50901,13108,9636,7146,7398,145,214
51621,50902,.,.,.,.,.,.
51624,50903,10159,9035,3579,3087,184,247
51628,51001,8573,8097,1879,1773,312,339


In [ ]:
rentas_muni = rentas_muni.rename(columns = {'cod_aux': 'cod_mun'})

In [ ]:
rentas_muni

,cod_mun,renta_salario_2018,renta_salario_2015,renta_pensiones_2018,renta_pensiones_2015,renta_desempleo_2018,renta_desempleo_2015
0,11903,4683,,2017,,320,
4,14901,3669,,2267,,846,
7,14902,4288,,2201,,846,
10,18077,4170,,3049,,813,
13,18916,4422,,2095,,503,
...,...,...,...,...,...,...,...
51618,50901,13108,9636,7146,7398,145,214
51621,50902,.,.,.,.,.,.
51624,50903,10159,9035,3579,3087,184,247
51628,51001,8573,8097,1879,1773,312,339


In [ ]:
rentas_muni.to_csv('/content/drive/MyDrive/Proyecto_KeepCoding (1)/Data/Dist_renta_hogares_municipios_15_18.txt', sep = ',', index = False)